# Tutorial: Building and Using AI Agents with Azure

Welcome to the **AI Agents Tutorial**! In this notebook, you'll learn how to build and use AI agents leveraging Azure services. We'll walk through the setup, necessary imports, function definitions, and integration with Azure's AI Project Client.

## Table of Contents
1. [Prerequisites](#Prerequisites)
2. [Importing Necessary Libraries](#Importing-Necessary-Libraries)
3. [Utility Functions](#Utility-Functions)
4. [Setting Up AI Project Client](#Setting-Up-AI-Project-Client)
5. [Defining the AI Agent](#Defining-the-AI-Agent)
6. [Helper Functions](#Helper-Functions)
7. [Running the AI Agent](#Running-the-AI-Agent)
8. [Conclusion](#Conclusion)

## Prerequisites

Before we begin, ensure you have the following installed:

- Python 3.8 or higher
- [Jupyter Notebook](https://jupyter.org/install)
- Azure SDK for Python
- Required Python packages (listed in the [Installation](#Installation) section)

### Installation

Install the necessary Python packages using `pip`. It's recommended to use a virtual environment.

```bash
pip install asyncio aiohttp beautifulsoup4 readability-lxml httpx dotenv azure-ai-projects azure-identity azure-monitor-opentelemetry opentelemetry-api promptflow
```

In [ ]:
%pip install asyncio aiohttp beautifulsoup4 readability-lxml httpx dotenv azure-ai-projects azure-identity azure-monitor-opentelemetry opentelemetry-api promptflow

## Importing Necessary Libraries

We'll start by importing all the necessary libraries and modules required for our AI agent.

In [2]:
# ------------------------------------
# Importing Libraries
# ------------------------------------

import asyncio
import os
import sys
from typing import Any, Callable, Set, Dict, List, Optional
import urllib, urllib.parse
from httpx import AsyncClient, HTTPStatusError, RequestError
import json
import datetime
import re

import aiohttp
from bs4 import BeautifulSoup
from readability import Document

import time
from pathlib import Path

from dotenv import load_dotenv

from azure.ai.projects.aio import AIProjectClient
from azure.ai.projects.models import (
    AsyncFunctionTool,
    RequiredFunctionToolCall,
    SubmitToolOutputsAction,
    ToolOutput,
    AsyncToolSet,
    CodeInterpreterTool,
    BingGroundingTool
)
from azure.ai.projects.telemetry.agents import AIAgentsInstrumentor
from azure.identity.aio import DefaultAzureCredential
from azure.monitor.opentelemetry import configure_azure_monitor

from opentelemetry import trace

## Utility Functions

### 1. Fetch Current Datetime

This function retrieves the current datetime and formats it as a JSON string.

In [3]:
def fetch_current_datetime(format: Optional[str] = None) -> str:
    """
    Get the current time as a JSON string, optionally formatted.
    
    :param format (Optional[str]): The format in which to return the current time. Defaults to None, which uses a standard format.
    :return: The current time in JSON format.
    :rtype: str
    """
    current_time = datetime.datetime.now()
    
    # Use the provided format if available, else use a default format
    if format:
        time_format = format
    else:
        time_format = "%Y-%m-%d %H:%M:%S"
    
    time_json = json.dumps({"current_time": current_time.strftime(time_format)})
    return time_json

### 2. Readability Check

A placeholder function to determine if the parsed HTML content is readable.

In [4]:
def is_probably_readable(soup: BeautifulSoup, min_score: int = 100) -> bool:
    try:
        doc = Document(str(soup))
        summary = doc.summary()
        if summary:
            return True
        else:
            return False
    except Exception:
        return False

### 3. Extract Readable Text

This asynchronous function uses the `readability` library to parse and extract the main content from HTML.

In [5]:
async def readable_text(params: Dict[str, Any]) -> Optional[str]:
    html = params['html']
    url = params['url']
    settings = params['settings']
    options = params.get('options', {})

    # Parse the HTML content
    soup = BeautifulSoup(html, 'html.parser')

    # Check if the document is probably readable
    if options.get('fallback_to_none') and not is_probably_readable(soup):
        return html

    # Use readability to parse the document
    doc = Document(html)
    parsed = doc.summary()
    parsed_title = doc.title()

    # Create a new BeautifulSoup object for the parsed content
    readability_soup = BeautifulSoup(parsed, 'html.parser')

    # Insert the title at the beginning of the content
    if parsed_title:
        title_element = readability_soup.new_tag('h1')
        title_element.string = parsed_title
        readability_soup.insert(0, title_element)

    return str(readability_soup)

### 4. Process HTML Content

This function processes the raw HTML content by removing specified elements and optionally transforming it to readable text.

In [6]:
async def process_html(html: str, url: str, settings: dict, soup: BeautifulSoup) -> str:
    body = soup.body
    if 'remove_elements_css_selector' in settings:
        for element in body.select(settings['remove_elements_css_selector']):
            element.decompose()
    
    simplified_body = body.decode_contents().strip()
    #simplified_body = re.sub(r'class="[^\"]*"', '', simplified_body)

    if isinstance(simplified_body, str):
        simplified = f"""<html lang="">
        <head>
            <title>
                {soup.title.string if soup.title else ''}
            </title>
        </head>
        <body>
            {simplified_body}
        </body>
    </html>"""
    else:
        simplified = html or ''
    
    ret = None
    if settings.get('html_transformer') == 'readableText':
        try:
            ret = await readable_text({'html': simplified, 'url': url, 'settings': settings, 'options': {'fallback_to_none': False}})
        except Exception as error:
            print(f"Processing of HTML failed with error: {error}")
            # Consider re-raising the exception or returning a default value
            # raise error
            ret = simplified  # Or some other default value
    
    return ret or simplified

### 5. Fetch Webpage Content

This asynchronous function retrieves the content of a webpage given its URL.

In [7]:
async def get_webpage(url: str) -> str:
    """Sends an HTTP GET request to the specified URI and returns the response body as a string.
    
    :param url (str): webpage url.
    :return: A string containing webpage content. 
    :rtype: str
    """
    """Returns the content of the webpage at the specified URL."""
    if not url:
        raise SystemError("url cannot be `None` or empty")
    
    
    async with aiohttp.ClientSession() as session:
        try:
            async with session.get(url, raise_for_status=True) as response:
                response_text = await response.text()
                result = await process_html(response_text,  url, {'html_transformer': 'readableText','readableTextCharThreshold': 500}, BeautifulSoup(response_text, 'html.parser'))
                return result
        except Exception as e:
            print(f"Error fetching or processing webpage {url}: {e}")
            return ""  # Or raise the exception, depending on desired behavior

### 6. Registering User Async Functions

Here, we define a set of asynchronous functions that can be used as tools by the AI agent.

In [10]:
# Statically defined user functions for fast reference with send_email as async but the rest as sync
user_async_function_tools: Set[Callable[..., Any]] = {
    fetch_current_datetime,
    get_webpage,
}

## Setting Up AI Project Client

Load environment variables and configure telemetry for monitoring.

In [11]:
import asyncio
import datetime
import os
import time
from pathlib import Path


dotenv_path = Path('deploy.env')
if dotenv_path.exists():
    load_dotenv(dotenv_path)
else:
    print("Error: 'deploy.env' file not found.  Make sure to create one.")

tracer = trace.get_tracer(__name__)

## Helper Functions

These functions assist in formatting the AI agent's responses, especially handling citations.

In [30]:
def print_response_with_citations(response):
    """Prints the response value with citations in Markdown format."""
    if 'value' not in response:
        print("Unexpected response format:", response)
        return "Error: Unexpected response format"

    value = response['value']
    annotations = response.get('annotations', [])

    last_index = 0
    output = ""

    for annotation in annotations:
        if annotation['type'] == 'url_citation':
            start_index = annotation['start_index']
            end_index = annotation['end_index']
            text = annotation['text']
            url = annotation['url_citation']['url']

            output += value[last_index:start_index]
            output += f"[{text}]({url})"
            last_index = end_index

    output += value[last_index:]
    return(output)

## Defining the AI Agent

The `agent_websearch` function is the core of our AI agent. It initializes the agent, sets up tools, creates communication threads, and processes runs.

In [19]:
async def agent_websearch(question: str, model_deployment: str, bing_grounding_conn: str) -> str:
    async with DefaultAzureCredential() as creds:
        async with AIProjectClient.from_connection_string(
            credential=creds, conn_str=os.environ["PROJECT_CONNECTION_STRING"],
        ) as project_client:
            
            application_insights_connection_string = await project_client.telemetry.get_connection_string()
            configure_azure_monitor(connection_string=application_insights_connection_string)
            
            bing_connection = await project_client.connections.get(connection_name=bing_grounding_conn)
            conn_id = bing_connection.id
            # Initialize assistant functions
            functions = AsyncFunctionTool(functions=user_async_function_tools)
            code_interpreter = CodeInterpreterTool()
            bing_grounding = BingGroundingTool(conn_id)

            
            toolset = AsyncToolSet()
            toolset.add(functions)
            toolset.add(code_interpreter)
            toolset.add(bing_grounding)
    
            agent_name = "docs-research-assistant"
            # Try to find an existing agent with the name "docs-research-assistant"
            agents = await project_client.agents.list_agents()
            agent = next((a for a in agents.data if a.name == agent_name), None)

            if agent is None:
                # Create agent if not found
                agent = await project_client.agents.create_agent(
                    model=model_deployment,
                    name=agent_name,
                    instructions='You are a question answerer using documentation site. Use the WebSearch tool to retrieve information to answer the questions from the docs site. Prepend "site:learn.microsoft.com" to any search query to search only the documentation site. You take in questions from a questionnaire and emit the answers, using documentation from the public web. You also emit links to any websites you find that help answer the questions. Do not address the user; make all responses solely in the third person. If you do not find information on a topic, simply respond that no information is available on that topic. The answer should be no greater than 1000 characters in length. When providing code examples, ensure they are properly formatted and include necessary context. If a question is ambiguous, ask for clarification.',
                    tools=functions.definitions + code_interpreter.definitions + bing_grounding.definitions
                )
                print(f"Created agent, agent ID: {agent.id}")
            else:
                print(f"Found existing agent: {agent.id}")

            # Create thread for communication
            thread = await project_client.agents.create_thread()
            print(f"Created thread, ID: {thread.id}")

            # Create and send message
            message = await project_client.agents.create_message(
                thread_id=thread.id, role="user", content=f"Current date is {datetime.datetime.now().strftime('%Y-%m-%d')}. {question}"
            )
            print(f"Created message, ID: {message.id}")

            # Create and process agent run in thread with tools
            # [START create_and_process_run]
            run = await project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id, toolset=toolset)
            # [END create_and_process_run]
            print(f"Run finished with status: {run.status}")

            if run.status == "failed":
                print(f"Run failed: {run.last_error}")

            print(f"Run completed with status: {run.status}")

            # Fetch and log all messages
            messages = await project_client.agents.list_messages(thread_id=thread.id)
            print(f"Messages: {messages}")

            for file_path_annotation in messages.file_path_annotations:
                print(f"File Paths:")
                print(f"Type: {file_path_annotation.type}")
                print(f"Text: {file_path_annotation.text}")
                print(f"File ID: {file_path_annotation.file_path.file_id}")
                print(f"Start Index: {file_path_annotation.start_index}")
                print(f"End Index: {file_path_annotation.end_index}")
                file_name = Path(file_path_annotation.text).name
                await project_client.agents.save_file(
                    file_id=file_path_annotation.file_path.file_id, file_name=file_name
                )
                print(f"Saved image file to: {Path.cwd() / file_name}")

            last_message = messages.text_messages[0].text
            return last_message

## Running the AI Agent

To execute the AI agent, you can run the `agent_websearch` function with appropriate parameters. Here's an example of how to use it.

In [33]:
question = "What is the latest update on Azure OpenAI service?"
model_deployment = "gpt-4o"  # Replace with your actual model deployment name
bing_grounding_conn = "binggrounding"  # Replace with your actual Bing connection name

response = await agent_websearch(question, model_deployment, bing_grounding_conn)
print("\nAI Agent Response:")
print_response_with_citations(response)


c:\Users\jacwang\AppData\Local\anaconda3\envs\pyaml\Lib\collections\__init__.py:449: RuntimeWarning: coroutine 'main' was never awaited
  result = tuple_new(cls, iterable)


Found existing agent: asst_aEu9rC2i3CztUq71krwWoPwJ
Created thread, ID: thread_JyBw73QBHcLnPMpGETBkY2uT
Created message, ID: msg_gmnsbiik8Qmjbcmc59m9WvOP
Run finished with status: RunStatus.COMPLETED
Run completed with status: RunStatus.COMPLETED
Messages: {'object': 'list', 'data': [{'id': 'msg_RnnLGZwkLY6OMxr8FfBGMewd', 'object': 'thread.message', 'created_at': 1741297403, 'assistant_id': 'asst_aEu9rC2i3CztUq71krwWoPwJ', 'thread_id': 'thread_JyBw73QBHcLnPMpGETBkY2uT', 'run_id': 'run_agaJ6Q6fUEwk9AjSMOdDnM1Y', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'As of February 2025, Azure OpenAI Service introduced the GPT-4.5 Preview, which excels in diverse text and image tasks. This latest model requires registration for access, and eligibility is determined by Microsoft. Additionally, the service has enhanced its Stored Completions API, which captures conversation history for use in evaluations and fine-tuning【3†source】.', 'annotations': [{'type': 'url_citation', 't

'As of February 2025, Azure OpenAI Service introduced the GPT-4.5 Preview, which excels in diverse text and image tasks. This latest model requires registration for access, and eligibility is determined by Microsoft. Additionally, the service has enhanced its Stored Completions API, which captures conversation history for use in evaluations and fine-tuning[【3†source】](https://azurecharts.com/updates).'

### Explanation:

- **question**: The query you want the AI agent to answer.
- **model_deployment**: The name of your deployed AI model.
- **bing_grounding_conn**: The name of your Bing grounding connection for web searches.

Ensure that you have set up your Azure credentials and necessary environment variables in the `deploy.env` file.

## Conclusion

In this tutorial, you've learned how to set up and use an AI agent with Azure services. By following the structured approach—importing libraries, defining utility functions, setting up the AI Project Client, and defining the agent—you can build robust AI-driven applications.

Feel free to extend this notebook by adding more functionalities, customizing the agent's instructions, or integrating additional tools as per your project requirements.